In [14]:
# Import Files
#%pylab inline

from __future__ import print_function
import numpy as np
import os
import sys
import numpy as np
import pandas as pd
from IPython.display import display, Image
from scipy import ndimage
from six.moves import cPickle as pickle
from sklearn.metrics import accuracy_score
import tensorflow as tf

# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

In [15]:
# Training Dataset Builder: Get the pixels, prepross them store the dataset using pickle files into the disk

image_size = 28

def image_pixel_standarize(image_pixel_val):
    return(image_pixel_val - 255.0/2)/255.0
    

def bld_pixels_arr(folder_name, min_no_of_image, max_no_of_image=None):
    # Build the pickle file for one Alphabet Image
    if max_no_of_image:
        image_filenames = os.listdir(folder_name)[0:max_no_of_image]
    else:
        image_filenames = os.listdir(folder_name)

    # Assign a numpy multudimensional array to store the image pixels, the image is 28*28 pixels
    dataset = np.ndarray(shape=(len(image_filenames), 
                                image_size, 
                                image_size
                               ),
                         dtype=np.float32)
    print (dataset.shape)
    for num_images, image in enumerate(image_filenames):
#         print (num_images)
        image_file_dir = os.path.join(folder_name, image)
        try:
            image_pixels = ndimage.imread(image_file_dir).astype(float)
            image_standarized = image_pixel_standarize(image_pixels)

            if image_standarized.shape != (image_size, image_size):
                raise Exception('Unexpected image shape: %s' % str(image_standarized.shape))
            dataset[num_images, :, :] = image_standarized
        except IOError as e:
            print('Could not read:', image, ':', e, '- hence skipping.')
        
    dataset = dataset[0:num_images+1, :, :]  # Will combine all the image pixels for 1 type Example All the 'a'
#     print (dataset.shape)
#     print (dataset)
    if num_images < min_no_of_image:
        raise Exception('Many fewer images than expected: %d < %d' %(num_images, min_no_of_image))

    print('Complete Training/Crossvalidation dataset :', dataset.shape)
    print('Mean:', np.mean(dataset))
    print('Standard deviation:', np.std(dataset))    
    return dataset


        
def crt_dataset(data_folders, dataset_type, min_no_of_image, max_no_of_image=None, force=None):    
    dataset_dir_n_names = []
    # One folder would comtain all the images pertaining to one particular alphabet.
    for folder in data_folders:  # [0:2]  
        print ('')
        print (folder)
        print ('Running for folder',os.path.basename(folder))
        set_filename = dataset_type + os.path.basename(folder) + '.pickle'
#         print (set_filename)
        dataset_dir_n_names.append(set_filename)
#         Ckech if the pickle file already exists, if not then we create one for each
        if os.path.exists(set_filename) and not force:
            print ('Yes+Yes--The pickle file exists')
        else:
            dataset = bld_pixels_arr(folder, min_no_of_image=min_no_of_image, max_no_of_image=max_no_of_image)
            try:
                with open(set_filename, 'wb') as f:
                    pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
            except Exception as e:
                print('Unable to save data to', set_filename, ':', e)
    return dataset_dir_n_names
                

DataPath_train = "/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Tensor-Flow-Learn/DataFolder/Training/"
data_folders_train = [os.path.join(DataPath_train,folders) for folders in os.listdir(DataPath_train) if folders!='.DS_Store']    
print (data_folders_train)

training_dataset_dir = '/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Tensor-Flow-Learn/DataPrepared/Training/'
# test_dataset_dir = "/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Udacity/Datapreparation/test_dataset/"

training_data_names_n_dir = crt_dataset(data_folders=data_folders_train, 
                            dataset_type=training_dataset_dir, 
                            min_no_of_image=4, 
                            max_no_of_image=None, 
                            force=None)
# test_data_names_n_dir = crt_dataset(data_folders=data_folders_test, 
#                             dataset_type=test_dataset_dir, 
#                             min_no_of_image=1800, 
#                             max_no_of_image=None, 
#                             force=None)

['/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Tensor-Flow-Learn/DataFolder/Training/a', '/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Tensor-Flow-Learn/DataFolder/Training/b', '/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Tensor-Flow-Learn/DataFolder/Training/c']

/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Tensor-Flow-Learn/DataFolder/Training/a
Running for folder a
Yes+Yes--The pickle file exists

/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Tensor-Flow-Learn/DataFolder/Training/b
Running for folder b
Yes+Yes--The pickle file exists

/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Tensor-Flow-Learn/DataFolder/Training/c
Running for folder c
Yes+Yes--The pickle file exists


In [16]:
# Create training and crossvalidation dataset 
def initialize_arrays(no_of_images, img_size):
    dataset = np.ndarray((no_of_images, img_size, img_size), dtype=np.float32)
    labels = np.ndarray(no_of_images, dtype=np.int32)
    return dataset, labels

def train_valid_test_builder(training_data_names_n_dir, training_size, crossvalid_size=0):
    no_of_labels = len(training_data_names_n_dir)
    training_dataset, training_labels = initialize_arrays(training_size, image_size)
    if crossvalid_size!=0:
        crossvalid_dataset, crossvalid_labels = initialize_arrays(crossvalid_size, image_size)
    else:
        crossvalid_dataset, crossvalid_labels = None, None

    training_size_per_class = training_size // no_of_labels
    crossvalid_size_per_class = crossvalid_size // no_of_labels
    print ('training/teting size per class = ',training_size_per_class)
    print ('crossvalid size per class = ', crossvalid_size_per_class)
    
    start_v, start_t = 0, 0
    end_v, end_t = crossvalid_size_per_class, training_size_per_class
    end_vt = crossvalid_size_per_class+training_size_per_class

    for label, pickle_files in enumerate(training_data_names_n_dir):      
#         print ('Running for alphabet: ', os.path.basename(pickle_files))
        try:
            with open(pickle_files, 'rb') as f:
                letter_data = pickle.load(f)
                np.random.shuffle(letter_data)          # This will suffle the data which will give the top elements as random
                if crossvalid_dataset is not None:
                    crossvalid_for_letter = letter_data[0:crossvalid_size_per_class, :, :]
                    crossvalid_dataset[start_v:end_v, :, :] = crossvalid_for_letter
                    crossvalid_labels[start_v:end_v] = label
                    start_v += crossvalid_size_per_class
                    end_v += crossvalid_size_per_class

                training_for_letter = letter_data[crossvalid_size_per_class:end_vt, :, :]
                training_dataset[start_t:end_t, :, :] = training_for_letter
                training_labels[start_t:end_t] = label
                start_t += training_size_per_class
                end_t += training_size_per_class
        except Exception as e:
            print('Unable to process data from', pickle_files, ':', e)
            raise
    return crossvalid_dataset, crossvalid_labels, training_dataset, training_labels



# Load the pixel Data Stored
training_size = 27      # Total Train size including all different labels
crossvalid_size = 9     # Total Crossvalid size including all different labels
batch_size = 12

print (training_data_names_n_dir)
crossvalid_dataset, crossvalid_labels, training_dataset, training_labels = \
train_valid_test_builder(training_data_names_n_dir, training_size, crossvalid_size)



['/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Tensor-Flow-Learn/DataPrepared/Training/a.pickle', '/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Tensor-Flow-Learn/DataPrepared/Training/b.pickle', '/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Tensor-Flow-Learn/DataPrepared/Training/c.pickle']
training/teting size per class =  9
crossvalid size per class =  3


In [17]:
complete_dataset_path = '/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Tensor-Flow-Learn/DataPrepared/dataset_complete.p'

try:
    f = open(complete_dataset_path, 'wb')
    dataset_complete = {
        'training_dataset': training_dataset,
        'training_labels': training_labels,
        'crossvalid_dataset': crossvalid_dataset,
        'crossvalid_labels': crossvalid_labels,
#         'test_dataset': test_dataset,
#         'test_labels': test_labels,
    }
    pickle.dump(dataset_complete, f, pickle.HIGHEST_PROTOCOL)
    f.close()
except Exception as e:
    print('Unable to save data to', complete_dataset_path, ':', e)
    raise

    
statinfo = os.stat(complete_dataset_path)
print('Compressed pickle size:', statinfo.st_size)

Compressed pickle size: 113426
